# Reference Forecasts

This [Jupyter notebook](https://jupyter.org) is designed to introduce you to the [Solar Forecast Arbiter's](https://solarforecastarbiter.org/) built-in reference forecast capabilities. It is divided into 2 sections:

1. [NWP-based forecasts](#NWP-based-forecasts)
2. [Persistence forecasts](#persistence-forecasts)

The API documentation is available [here](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html).

Click the ">| Run" button in the toolbar above or type shift-enter to run the code in each cell. The help menu contains a brief User Interface Tour.

In [1]:
import datetime
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd

from bokeh.core.properties import value
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Category10_10 as PALETTE
TOOLS = "pan,box_zoom,xwheel_zoom,reset,save"
output_notebook()

Loading BokehJS ...

In [2]:
from solarforecastarbiter import datamodel

## NWP-based forecasts

Forecasts based on NWP model data are used for intraday and longer forecasts. The Solar Forecast Arbiter contains a set of functions to process data from NWP forecasts. Here, we explore some of the functionality with an emphasis on obtaining results. See the [NWP section of the documentation](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html#nwp) for additional information.

The `solarforecastarbiter` package includes a handful of subsetted NWP model runs for testing. We'll use these for our demonstration below. The models were all initialized at 2019-05-15 00Z and the subsets include about half a degree of latitude and longitude near Tucson, AZ.

In [3]:
from solarforecastarbiter.io import nwp
# find the files
base_path = Path(nwp.__file__).resolve().parents[0] / 'tests/data'
# define file loading function that knows where to find the files
load_forecast = partial(nwp.load_forecast, base_path=base_path)

In [4]:
# define coordinates
latitude = 32.2
longitude = -110.9
elevation = 700

# define initialization time, forecast start time, forecast end time
init_time = pd.Timestamp('20190515T0000Z')
start = pd.Timestamp('20190515T0100Z')
end = pd.Timestamp('20190518T0000Z')

The Solar Forecast Arbiter organizes its metadata and data according to the ideas described in the [data model](https://solarforecastarbiter.org/datamodel/). The `solarforecastarbiter` package implements this data model in `solarforecastarbiter.datamodel`. Here, we define a `Site` object for which to make forecasts.

In [5]:
# define a Site object
site = datamodel.Site(
    name='Tucson, AZ',
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    timezone='America/Phoenix'
)

Next we select a NWP+post processing model, load and process the data, and plot the processed data.

Here we use the subhourly irradiance directly from the HRRR model.

In [6]:
# import the relevant python modules
from solarforecastarbiter.reference_forecasts import main, models

# select the model
model = models.hrrr_subhourly_to_subhourly_instantaneous

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data. ac_power is None because site is not a power plant.
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

In [7]:
ghi.head()

time
2019-05-15 01:00:00+00:00    212.0
2019-05-15 01:15:00+00:00    162.0
2019-05-15 01:30:00+00:00    104.0
2019-05-15 01:45:00+00:00     66.0
2019-05-15 02:00:00+00:00     26.0
Freq: 15T, Name: ghi, dtype: float32

Here's a function that will make a plot of the processed NWP data. The details of the function are not important for this exercise.

In [8]:
def plot_nwp(ghi, dni, dhi, air_temperature, wind_speed, ac_power):
    """Make a plot of the variables from processed NWP forecasts"""
    # data prep
    if ac_power is None:
        ac_power = pd.Series(None, index=ghi.index)
    data = dict(zip(('GHI', 'DNI', 'DHI', 'Air temperature', 'Wind speed', 'AC power', 'index'),
                    (ghi, dni, dhi, air_temperature, wind_speed, ac_power, ghi.index)))
    source = ColumnDataSource(data)

    figs = []
    fig_kwargs = dict(tools=TOOLS, x_axis_type="datetime", plot_height=200)
    palette = iter(PALETTE)
    
    # irradiance figure
    fig1 = figure(**fig_kwargs)
    for name in ('GHI', 'DNI', 'DHI'):
        fig1.line(x='index', y=name, source=source, legend=value(name), color=next(palette), line_width=2)
    fig1.yaxis.axis_label = "Irradiance (W/m^2)"
    fig1.legend.location = "top_left"
    figs.append(fig1)

    # air temperature figure
    fig_kwargs['x_range'] = fig1.x_range  # link x-zoom
    fig2 = figure(**fig_kwargs)
    fig2.line(x='index', y='Air temperature', source=source, legend=value("Air temperature"), color=next(palette), line_width=2)
    fig2.yaxis.axis_label = "Air temperature (C)"
    fig2.legend.location = "top_left"
    figs.append(fig2)

    # wind speed figure
    fig3 = figure(**fig_kwargs)
    fig3.line(x='index', y='Wind speed', source=source, legend=value("Wind speed"), color=next(palette), line_width=2)
    fig3.yaxis.axis_label = "Wind speed (m/s)"
    fig3.legend.location = "top_left"
    figs.append(fig3)

    # ac power figure
    if not ac_power.dropna().empty:
        fig4 = figure(**fig_kwargs)
        fig4.line(x='index', y='AC power', source=source, legend=value("AC power"), color=next(palette), line_width=2)
        fig4.yaxis.axis_label = "AC power (MW)"
        fig4.legend.location = "top_left"
        figs.append(fig4)
    
    figs[-1].xaxis.axis_label = 'Time (UTC)'

    grid = gridplot(figs, ncols=1, plot_width=800)
    return grid

Use the function to plot the HRRR data.

In [9]:
grid = plot_nwp(ghi, dni, dhi, air_temperature, wind_speed, ac_power)
show(grid)

The Solar Forecast Arbiter includes processing functions derive GHI from a NWP model's cloud cover forecast and estimate the DNI and DHI using the [Erbs model](https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.erbs.html?highlight=erbs). These functions are applied to data from the GFS, NAM, and RAP weather models. 

For example, GFS data is processed as follows:

1. Load time series forecast of cloud cover, air temperature, and wind speed from gridded data.
2. Convert mixed interval cloud cover in raw data into standard intervals.
2. Interpolate cloud cover, air temperature, wind speed to 5 minute resolution to minimize errors due to solar position.
2. Compute GHI from cloud cover.
2. Compute DNI and DHI from GHI.
2. If required, calculate AC power.
2. Resample 5 minute data to 1 hour data.

See the `models` [documentation](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.models.html#module-solarforecastarbiter.reference_forecasts.models) for more explanation. The plot below shows the results of these processing steps.

In [10]:
# select the model
model = models.gfs_quarter_deg_to_hourly_mean

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data. ac_power is None because site is not a power plant.
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

grid = plot_nwp(ghi, dni, dhi, air_temperature, wind_speed, ac_power)
show(grid)

The forecast function will produce an AC power forecast if supplied with the appropriate metadata. Here we define a single axis tracker.

In [11]:
# define the modeling parameters
modeling_parameters = datamodel.SingleAxisModelingParameters(
    ac_capacity=10,  # always in MW
    dc_capacity=13,
    temperature_coefficient=-0.003,  # 1/C
    dc_loss_factor=0,
    ac_loss_factor=0,
    axis_tilt=0,
    axis_azimuth=0,
    ground_coverage_ratio=0.4,
    backtrack=True,
    max_rotation_angle=50,
)
# define a SolarPowerPlant object
plant = datamodel.SolarPowerPlant(
    name='Tucson AZ Plant',
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    timezone='America/Phoenix',
    modeling_parameters=modeling_parameters
)

Now we run the same code as before to create an AC power forecast from the GFS model.

In [12]:
# select the model
model = models.gfs_quarter_deg_to_hourly_mean

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data. only difference from before is plant replaces site
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(plant, model_wrapped, init_time, start, end)

grid = plot_nwp(ghi, dni, dhi, air_temperature, wind_speed, ac_power)
show(grid)

## Persistence forecasts

The solarforecastarbiter supports several varieties of persistence forecasts:

1. Persistence of observed values
2. Persistence of irradiance or power accounting for solar position. (This is sometimes referred to as smart persistence, but we discourage that term because it's ambiguous.)

The [`persistence`](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.persistence.html#module-solarforecastarbiter.reference_forecasts.persistence) module contains functions that implement these methods and are easily adapted to a user's own data and workflow. The [`reference_forecasts.main.run_persistence`](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.main.run_persistence.html#solarforecastarbiter.reference_forecasts.main.run_persistence) function is a higher-level interface that integrate more tightly with the Solar Forecast Arbiter data model and API.

### run_persistence

The code below queries the API for observation metadata and data. We'll use the NREL MIDC OASIS station located on the University of Arizona campus. See the [Data Upload and Download]() exercises for more information on the data transfer.

In [13]:
from solarforecastarbiter.io.api import APISession, request_cli_access_token
# don't store your real passwords or tokens in plain text like this! only for demonstration purposes!
token = request_cli_access_token('testing@solarforecastarbiter.org', 'Thepassword123!')
session = APISession(token)
observations = session.list_observations()
observations_oasis = list(filter(lambda x: 'NREL MIDC' in x.site.name and 'Arizona' in x.site.name, observations))
oasis_ghi = observations_oasis[0]
oasis_ghi

Observation(name='University of Arizona OASIS ghi', variable='ghi', interval_value_type='interval_mean', interval_length=Timedelta('0 days 00:01:00'), interval_label='ending', site=Site(name='NREL MIDC University of Arizona OASIS', latitude=32.22969, longitude=-110.95534, elevation=786.0, timezone='Etc/GMT+7', site_id='9f61b880-7e49-11e9-9624-0a580a8003e9', provider='Reference', extra_parameters='{"network": "NREL MIDC", "network_api_id": "UAT", "network_api_abbreviation": "UA OASIS", "observation_interval_length": 1}'), uncertainty=0.0, observation_id='9f657636-7e49-11e9-b77f-0a580a8003e9', extra_parameters='{"network": "NREL MIDC", "network_api_id": "UAT", "network_api_abbreviation": "UA OASIS", "observation_interval_length": 1, "network_data_label": "Global Horiz (platform) [W/m^2]"}', units='W/m^2')

In [14]:
start = pd.Timestamp('20190515 0000Z')
end = pd.Timestamp('20190518 0000Z')
oasis_ghi_values = session.get_observation_values(oasis_ghi.observation_id, start, end)
oasis_ghi_values.head()

,value,quality_flag
timestamp,,
2019-05-15 00:00:00+00:00,428.807,2
2019-05-15 00:01:00+00:00,425.372,2
2019-05-15 00:02:00+00:00,422.223,2
2019-05-15 00:03:00+00:00,418.949,2
2019-05-15 00:04:00+00:00,414.445,2


Our example persistence forecast will be a 15 minute ahead, 15 minute interval average forecast for the NREL MIDC OASIS GHI observation. Both the observation's and forecast's intervals are labeled by their ending time.

In [15]:
forecast = datamodel.Forecast(
    name='15 min ahead, 15 min avg', 
    issue_time_of_day=datetime.time(0), 
    lead_time_to_start=pd.Timedelta('15min'),
    interval_length=pd.Timedelta('15min'),
    run_length=pd.Timedelta('15min'),
    interval_label='ending',
    interval_value_type='interval_mean',
    variable='ghi',
    site=oasis_ghi
)

Resample the observed 1 minute interval data to have the same interval length as the forecasts for a fair comparison.

In [16]:
resampled_data = oasis_ghi_values.resample(forecast.interval_length, label='right').mean()
resampled_data['quality_flag'] = 0

Define a forecast `run_time` and `issue_time`. For operational forecasts, `run_time` is typically set to now. For retrospective forecasts, `run_time` is the time by which the forecast should be run so that it could have been be delivered for the `issue_time`. Forecasts will only use data with timestamps before `run_time`. Let's assume they are the same for this exercise.

In [17]:
run_time = pd.Timestamp('20190515 1900Z')
issue_time = pd.Timestamp('20190515 1900Z')

Let's look at the observation values leading up to the forecast run time. We expect the persistence forecast to be equal to an average of these values.

In [18]:
print(oasis_ghi_values.loc[run_time - forecast.interval_length:run_time])
print('\n mean:')
print(oasis_ghi_values.loc[run_time - forecast.interval_length:run_time].mean())

                              value  quality_flag
timestamp                                        
2019-05-15 18:45:00+00:00  1059.300             2
2019-05-15 18:46:00+00:00  1059.250             2
2019-05-15 18:47:00+00:00  1058.190             2
2019-05-15 18:48:00+00:00  1059.600             2
2019-05-15 18:49:00+00:00  1059.650             2
2019-05-15 18:50:00+00:00  1062.410             2
2019-05-15 18:51:00+00:00  1043.830             2
2019-05-15 18:52:00+00:00  1053.610             2
2019-05-15 18:53:00+00:00  1082.200             2
2019-05-15 18:54:00+00:00  1087.860             2
2019-05-15 18:55:00+00:00  1097.610             2
2019-05-15 18:56:00+00:00  1104.210             2
2019-05-15 18:57:00+00:00  1069.280             2
2019-05-15 18:58:00+00:00   897.564             2
2019-05-15 18:59:00+00:00   832.101             2
2019-05-15 19:00:00+00:00   916.081             2

 mean:
value           1033.921625
quality_flag       2.000000
dtype: float64


In [19]:
oasis_ghi_fx = main.run_persistence(session, oasis_ghi, forecast, run_time, issue_time, index=False)
oasis_ghi_fx

2019-05-15 19:30:00+00:00    1033.921625
Freq: 15T, dtype: float64

It works! 

There is a problem, though. A 15 minute interval should include *either* the start time, value pair *or* the end time, value pair, but *not* both. This bug is tracked in [this GitHub issue](https://github.com/SolarArbiter/solarforecastarbiter-core/issues/80). The resampled data that we'll use for plotting actually shows the correct result. The correct result is slightly below than the incorrect result because it discards the above average value that marks the end of the last interval.

In [20]:
resampled_data.loc[run_time - forecast.interval_length]

value           1032.537933
quality_flag       0.000000
Name: 2019-05-15 18:45:00+00:00, dtype: float64

To make a [forecast evaluation time series](https://solarforecastarbiter.org/usecases/#forecastevalts), we need to loop over many run times.

In [21]:
run_times = pd.date_range(start=start, end=end, freq=forecast.run_length)

In [22]:
oasis_ghi_fx_runs = []
for run_time in run_times:
    issue_time = run_time  # an offset may be needed in an operational environment
    fx_run = main.run_persistence(session, oasis_ghi, forecast, run_time, issue_time, index=False)
    oasis_ghi_fx_runs.append(fx_run)
oasis_ghi_fx = pd.concat(oasis_ghi_fx_runs)

In [23]:
def plot_persistence(observation_values, forecast_values=None, forecast_values_index=None):
    fig_kwargs = dict(tools=TOOLS, x_axis_type="datetime", plot_height=250, plot_width=800)
    palette = iter(PALETTE)
    fig1 = figure(**fig_kwargs)
    fig1.line(observation_values.index, observation_values['value'], legend='observed', color='black', line_width=2)
    color = next(palette)
    if forecast_values is not None:
        fig1.line(forecast_values.index, forecast_values, legend='persistence', color=color, line_width=2)
    color = next(palette)
    if forecast_values_index is not None:
        fig1.line(forecast_values_index.index, forecast_values_index, legend='persistence index', color=color, line_width=2)
    fig1.yaxis.axis_label = "Irradiance (W/m^2)"
    fig1.legend.location = "top_left"
    fig1.xaxis.axis_label = 'Time (UTC)'
    return fig1

In [24]:
fig1 = plot_persistence(resampled_data, oasis_ghi_fx)
show(fig1)

As expected, the persistence forecast appears to be a simple translation in time of the input data.

Next we will examine forecasts using persistence of clear sky index to account for changing solar position.

In [25]:
oasis_ghi_fx_runs = []
for run_time in run_times:
    issue_time = run_time  # an offset may be needed in an operational environment
    fx_run = main.run_persistence(session, oasis_ghi, forecast, run_time, issue_time, index=True)
    oasis_ghi_fx_runs.append(fx_run)
oasis_ghi_index_fx = pd.concat(oasis_ghi_fx_runs)

In [26]:
fig1 = plot_persistence(resampled_data, forecast_values_index=oasis_ghi_index_fx)
show(fig1)

The persistence of clear sky index accounts for the change in solar position. It also introduces errors near sunrise when the ratio of actual to expected clear sky depends sensitively on the instrumentation and the clear sky model.

"Day ahead persistence" is another flavor of persistence in which the observed values for a 24 hour period are used as the forecast for another 24 hour period. The Solar Forecast Arbiter's [`persistence_interval`](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.persistence.persistence_interval.html#solarforecastarbiter.reference_forecasts.persistence.persistence_interval) function implements this feature. The high-level interface currently does not work with the function due to the same [bug](https://github.com/SolarArbiter/solarforecastarbiter-core/issues/80) discussed above.

In [27]:
da_forecast = datamodel.Forecast(
    name='day ahead persistence', 
    issue_time_of_day=datetime.time(7),  # local midnight
    lead_time_to_start=pd.Timedelta('0min'),
    interval_length=pd.Timedelta('1h'),
    run_length=pd.Timedelta('24hr'),
    interval_label='beginning',
    interval_value_type='interval_mean',
    variable='ghi',
    site=oasis_ghi
)

In [28]:
run_time = pd.Timestamp('20190516 0000-0700')
issue_time = run_time
try:
    fx_run = main.run_persistence(session, oasis_ghi, da_forecast, run_time, issue_time, index=False)
except ValueError as e:
    print(e)

Length of passed values is 25, index implies 24


### persistence module

This section demonstrates the lower level persistence functions.

In [29]:
from solarforecastarbiter.reference_forecasts import persistence

The functions in the persistence module are meant to be easily useable by the community. To facilitate this, the persistence functions allow a user to provide a function that will load their data rather than loading data from the Solar Forecast Arbiter API. Below, we define a function that will load the appropriate data.

In [30]:
# change default load_data function so we can use without API access
def load_data_base(data, observation, data_start, data_end):
    # slice doesn't care about closed or interval label 
    # so here we manually adjust start and end times
    if 'instant' in observation.interval_label:
        pass
    elif observation.interval_label == 'ending':
        data_start += pd.Timedelta('1s')
    elif observation.interval_label == 'beginning':
        data_end -= pd.Timedelta('1s')
    return data[data_start:data_end]

load_data = partial(load_data_base, oasis_ghi_values['value'])

The code below makes two forecasts, one for persistence of the observed GHI (`persistence_scalar`) and one using persistence of the clear sky index (`persistence_scalar_index`). Notice how the persistence index forecast accounts for the changing solar position.

In [31]:
observation = oasis_ghi

In [32]:
# persistence forecasts with scalar index
# observation.interval_label = 'ending', 
# so data ranges are exclusive of start and inclusive of end
data_start = pd.Timestamp('20190515 1500Z')
data_end = pd.Timestamp('20190515 1600Z')
forecast_start = pd.Timestamp('20190515 1600Z')
forecast_end = pd.Timestamp('20190515 1700Z')
interval_length = pd.Timedelta('5min')
interval_label = 'ending'

fx_per = persistence.persistence_scalar(
    observation, data_start, data_end, forecast_start,
    forecast_end, interval_length, interval_label, load_data=load_data)
fx_per_idx = persistence.persistence_scalar_index(
    observation, data_start, data_end, forecast_start,
    forecast_end, interval_length, interval_label, load_data=load_data)

resampled_data = oasis_ghi_values.resample(interval_length, label='right').mean()
fig = plot_persistence(resampled_data, fx_per, fx_per_idx)
show(fig)

Same data and forecast methods but for a cloudy period.

In [33]:
# persistence forecasts with scalar index
# observation.interval_label = 'ending', 
# so data ranges are exclusive of start and inclusive of end
data_start = pd.Timestamp('20190515 1700Z')
data_end = pd.Timestamp('20190515 1800Z')
forecast_start = pd.Timestamp('20190515 1800Z')
forecast_end = pd.Timestamp('20190515 1900Z')
interval_length = pd.Timedelta('5min')
interval_label = 'ending'

fx_per = persistence.persistence_scalar(
    observation, data_start, data_end, forecast_start,
    forecast_end, interval_length, interval_label, load_data=load_data)
fx_per_idx = persistence.persistence_scalar_index(
    observation, data_start, data_end, forecast_start,
    forecast_end, interval_length, interval_label, load_data=load_data)

resampled_data = oasis_ghi_values.resample(interval_length, label='right').mean()
fig = plot_persistence(resampled_data, fx_per, fx_per_idx)
show(fig)

The "day ahead persistence" function [`persistence_interval`](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.persistence.persistence_interval.html#solarforecastarbiter.reference_forecasts.persistence.persistence_interval) works here because our example `load_data` function accounts for interval labeling.

In [34]:
# persistence forecasts with scalar index
# observation.interval_label = 'ending', 
# so data ranges are exclusive of start and inclusive of end
data_start = pd.Timestamp('20190515 0600Z')
data_end = pd.Timestamp('20190516 0600Z')
forecast_start = pd.Timestamp('20190516 0600Z')
interval_length = pd.Timedelta('1hr')
interval_label = 'ending'

fx_per_interval = persistence.persistence_interval(
    observation, data_start, data_end, forecast_start,
    interval_length, interval_label, load_data=load_data)

resampled_data = oasis_ghi_values.resample(interval_length, label='right').mean()
fig = plot_persistence(resampled_data, fx_per_interval)
show(fig)